In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
import numpy as np
from tensorflow.keras.preprocessing import image

# Veri yolu tanımları
train_data_dir = 'flowers/train'
test_data_dir = 'flowers/test'
img_size = (64, 64)

# Veri ön işleme ve artırma
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Veri yükleyici
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

# VGG19 modelini yükleyin (ağırlıkları ImageNet üzerinde eğitilmiş olarak alır)
base_model = VGG19(input_shape=(64, 64, 3), include_top=False, weights='imagenet')

# CNN modeli oluşturun
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(17, activation='softmax'))  # 17 sınıf varsa

# VGG19 modelinin üst katmanlarını eğitmeyin
base_model.trainable = False

# Modeli derleme
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Modeli eğit
model.fit(train_generator,
          epochs=50,
          validation_data=test_generator)

# Test veri kümesinden bir örnek görsel seç
test_image_path = 'image_0805.jpg'

# Görseli modele uygun formata getir
test_img = image.load_img(test_image_path, target_size=(64, 64))
test_img_array = image.img_to_array(test_img)
test_img_array = np.expand_dims(test_img_array, axis=0)
test_img_array /= 255.0  # Veriyi normalize et

# Model tarafından yapılan tahmin
predictions = model.predict(test_img_array)

# Tahminlerden en yüksek olasılığa sahip sınıfı bul
predicted_class_index = np.argmax(predictions[0])
print("Tahmin Edilen Sınıf Index:", predicted_class_index)

# Tahmin edilen sınıf adı
class_labels = train_generator.class_indices
class_labels = dict((v, k) for k, v in class_labels.items())
predicted_class = class_labels[predicted_class_index]
print("Tahmin Edilen Sınıf:", predicted_class)

Found 1190 images belonging to 17 classes.
Found 168 images belonging to 17 classes.
Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 20s 427ms/step - accuracy: 0.1522 - loss: 2.8892 - val_accuracy: 0.4226 - val_loss: 1.9560
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 17s 421ms/step - accuracy: 0.3717 - loss: 2.0186 - val_accuracy: 0.5893 - val_loss: 1.5477
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 17s 414ms/step - accuracy: 0.5026 - loss: 1.6517 - val_accuracy: 0.6250 - val_loss: 1.4138
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 17s 420ms/step - accuracy: 0.5317 - loss: 1.5077 - val_accuracy: 0.6429 - val_loss: 1.2054
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 17s 414ms/step - accuracy: 0.5958 - loss: 1.3235 - val_accuracy: 0.6548 - val_loss: 1.0564
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 17s 417ms/step - accuracy: 0.5971 - loss: 1.2707 - val_accuracy: 0.6607 - val_loss: 1.0040
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 16s 410ms/step - accuracy: 0.6392 - loss: 1.1473 - val_accuracy: 0.6786 - val_loss: 1.0246
Epoch 8/50
3

38/38 ━━━━━━━━━━━━━━━━━━━━ 17s 414ms/step - accuracy: 0.8577 - loss: 0.3826 - val_accuracy: 0.7321 - val_loss: 1.0248
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
Tahmin Edilen Sınıf Index: 6
Tahmin Edilen Sınıf: daisy
